<a href="https://colab.research.google.com/github/2025airda-max/---/blob/main/fast_whisper_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title 🔧 Установка (запусти один раз)
!pip install -q faster-whisper
!pip install -q gradio  # для удобного интерфейса

print("✅ Установка завершена!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.9 MB/s eta 0:00:00
✅ Установка завершена!


In [2]:
#@title 🚀 Загрузка модели Whisper Large-V3-Turbo
from faster_whisper import WhisperModel
import torch

# Проверяем GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"

print(f"🖥️ Устройство: {device.upper()}")
print("⏳ Загружаю модель large-v3-turbo (это займёт 1-2 минуты)...")

model = WhisperModel(
    "large-v3-turbo",
    device=device,
    compute_type=compute_type
)

print("✅ Модель загружена и готова к работе!")

🖥️ Устройство: CUDA
⏳ Загружаю модель large-v3-turbo (это займёт 1-2 минуты)...


model.bin:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

vocabulary.json: 0.00B [00:00, ?B/s]

✅ Модель загружена и готова к работе!


In [4]:
#@title 📝 Функция транскрибации

def transcribe_audio(audio_path, language=None, task="transcribe"):
    """
    Транскрибирует аудио/видео файл.

    Args:
        audio_path: путь к файлу
        language: язык ('ru', 'en', 'uk' и т.д.) или None для автоопределения
        task: 'transcribe' (транскрибация) или 'translate' (перевод на английский)
    """
    print(f"🎙️ Обрабатываю: {audio_path}")
    print("⏳ Это может занять несколько минут...")

    segments, info = model.transcribe(
        audio_path,
        language=language,
        task=task,
        beam_size=5,
        vad_filter=True,  # убирает тишину
        vad_parameters=dict(min_silence_duration_ms=500)
    )

    print(f"📊 Обнаружен язык: {info.language} (вероятность: {info.language_probability:.1%})")
    print(f"⏱️ Длительность: {info.duration:.1f} сек\n")

    # Собираем результат
    full_text = []
    timestamps = []

    for segment in segments:
        full_text.append(segment.text.strip())
        timestamps.append({
            "start": segment.start,
            "end": segment.end,
            "text": segment.text.strip()
        })
        # Показываем прогресс
        print(f"[{segment.start:.1f}s → {segment.end:.1f}s] {segment.text.strip()}")

    return "\n".join(full_text), timestamps

print("✅ Функция готова!")

✅ Функция готова!


In [5]:
#@title 📤 Загрузи файл и получи транскрипцию
from google.colab import files
import os

# Загрузка файла
print("📤 Выбери аудио или видео файл (mp3, wav, mp4, m4a, webm и др.):")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    print(f"\n✅ Загружен: {filename}")

    #@markdown ---
    #@markdown ### Настройки:
    language = "ru" #@param ["ru", "en", "uk", "auto"] {allow-input: true}
    task = "transcribe" #@param ["transcribe", "translate"]

    # Если auto - передаём None для автоопределения
    lang = None if language == "auto" else language

    # Транскрибация
    text, timestamps = transcribe_audio(filename, language=lang, task=task)

    print("\n" + "="*50)
    print("📄 ПОЛНЫЙ ТЕКСТ:")
    print("="*50)
    print(text)

📤 Выбери аудио или видео файл (mp3, wav, mp4, m4a, webm и др.):


Выходные данные были обрезаны до нескольких последних строк (5000).
Ну, потому что, ну, как бы считается, что это, ну, может, некрасиво как-то, или вот каким-то последствием.
Скажите мне, ну, а если человек едет в транспорте в метро, и ему наступили на ногу, у него какое работает состояние, духовное или не духовное?
Может, он в это время молится, а может быть, он в это время дает страстное мечтание, тоже духовное.
Мечтание – это сфера метафизическая, следовательно, духовная.
Поэтому все дело в языке, как говорил этот отец наш Егенштейн, философ, он говорил про язык.
То есть то, что можно языком выразить, о том можно говорить, пусть нельзя выразить, лучше об этом молчать.
Мы будем этого правила просто придерживаться, ну, это так немножко философское отступление.
Что говорит психология насчет того, что гордыня, что такое, как с ней работать?
Это, скажем так, направление для повышения адаптивности человека, снижения его адаптивности к внешнему миру, снижения внутреннего конфликта.
То есть

In [6]:
#@title 💾 Сохранить результат в файл

#@markdown ### Формат сохранения:
save_format = "txt" #@param ["txt", "srt", "json"]

output_name = os.path.splitext(filename)[0]

if save_format == "txt":
    output_file = f"{output_name}_transcript.txt"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(text)

elif save_format == "srt":
    output_file = f"{output_name}_subtitles.srt"
    with open(output_file, "w", encoding="utf-8") as f:
        for i, seg in enumerate(timestamps, 1):
            start = seg["start"]
            end = seg["end"]
            # Форматируем время для SRT
            start_srt = f"{int(start//3600):02d}:{int((start%3600)//60):02d}:{int(start%60):02d},{int((start%1)*1000):03d}"
            end_srt = f"{int(end//3600):02d}:{int((end%3600)//60):02d}:{int(end%60):02d},{int((end%1)*1000):03d}"
            f.write(f"{i}\n{start_srt} --> {end_srt}\n{seg['text']}\n\n")

elif save_format == "json":
    import json
    output_file = f"{output_name}_transcript.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump({"text": text, "segments": timestamps}, f, ensure_ascii=False, indent=2)

print(f"✅ Сохранено: {output_file}")

# Скачиваем файл
files.download(output_file)

✅ Сохранено: Встреча в Телемосте 26.01.26 20-00-45 — запись_transcript.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>